# AWS Wrong implementation of the SCIM protocol

* Author: Gustavo Lichti Mendonça
* Mail: gustavo.lichti@gmail.com
* This Code: https://github.com/lichti/aws-iam-scim-problem

## Apology for cyberark

On November 6, 2021, I published a repository called [AWS-SSO-PROVISIONING](https://github.com/lichti/aws-sso-provisioning) criticizing CyberArk. However, on March 4, 2023, I forked it and created another repository called [aws-sso-google-sync](https://github.com/lichti/aws-sso-google-sync). While implementing the routine to [Remove or Empty deleted groups from Google Directory](https://github.com/lichti/aws-sso-google-sync?tab=readme-ov-file#remove-or-empty-deleted-groups-from-google-directory), I discovered a logic error in the [UpdateGroup](https://github.com/lichti/aws-sso-provisioning?tab=readme-ov-file#updategroup) method that prevented the routine from executing when `members` was an empty list. That day, after much head-scratching, I identified the mistake that I am now disclosing.

This disclosure comes in the form of an apology to CyberArk because one of the issues was precisely this, and had it not been for a logic error, I would have understood it, and the public criticism would not have existed.

# Deps, Config and Helpers

## Dependencies installing

In [ ]:
%%bash
pip install requests
pip install pyyaml

## Imports

In [ ]:
import requests
import json
import configparser
import time

## Load config file with credentials

Read more about configparser: https://docs.python.org/3/library/configparser.html

Config Teamplate:

```text
[AWS-SSO-SCIM]
base_url = https://scim.us-east-1.amazonaws.com/YOUR-AWS-SSO-ID/scim/v2/
bearertoken = YOUR-AWS-SSO-BEARERTOKEN
```

In [ ]:
config = configparser.ConfigParser()
config.read('aws-sso-scim.ini')

## SCIM AWS SSO CONFIG

Learn more about AWS SSO SCIM:
* https://docs.aws.amazon.com/singlesignon/latest/developerguide/supported-apis.html

In [ ]:
base_url = config['AWS-SSO-SCIM']['base_url']
bearertoken = config['AWS-SSO-SCIM']['bearertoken']
users_url = f"{base_url}Users"
headers_auth = {"Authorization": f"Bearer {bearertoken}", "Content-type": "application/json"}

## HTTP helpers

Basic http methods helpers (get, post, put, patch, delete)

Recommended reading: 
* https://datatracker.ietf.org/doc/html/rfc7231#section-4.3
* https://datatracker.ietf.org/doc/html/rfc7644#section-3.2

### Get

In [ ]:
def get(path=None, params=None):
    return requests.get(f"{base_url}{path}",headers=headers_auth, params=params)

### Post

In [ ]:
def post(path=None, params=None, data=None):
    return requests.post(f"{base_url}{path}",headers=headers_auth, data=data)

### Put

In [ ]:
def put(path=None, params=None, data=None):
    return requests.put(f"{base_url}{path}",headers=headers_auth, data=data)

### Patch

In [ ]:
def patch(path=None, params=None, data=None):
    return requests.patch(f"{base_url}{path}",headers=headers_auth, data=data)

### Delete

In [ ]:
def delete(path=None):
    return requests.delete(f"{base_url}{path}",headers=headers_auth)

## SCIM helpers

Basic SCIM methods helpers

Learn more abut SCIM:
* https://datatracker.ietf.org/doc/html/rfc7642
* https://datatracker.ietf.org/doc/html/rfc7643
* https://datatracker.ietf.org/doc/html/rfc7644
* https://openid.net/specs/fastfed-scim-1_0-02.html#rfc.section.4

### Users

#### CreateUser

https://datatracker.ietf.org/doc/html/rfc7644#section-3.3

In [ ]:
def createUser(userName=None,familyName=None,givenName=None,displayName=None,email=None,
               preferredLanguage="en-US",locale="en-US",timezone="America/Sao_Paulo",active=True):
    if userName and familyName and givenName and displayName and email:
        data = {
            "userName": f"{userName}",
            "name": {
                "familyName": f"{familyName}",
                "givenName": f"{givenName}",
            },
            "displayName": f"{displayName}",
            "emails": [
                {
                    "value": f"{email}",
                    "type": "work",
                    "primary": True
                }
            ],
            "preferredLanguage": f"{preferredLanguage}",
            "locale": f"{locale}",
            "timezone": f"{timezone}",
            "active": f"{active}",
        }
        res = post(path=f"Users", data=json.dumps(data))
        if res.status_code == 201:
            return json.loads(res.text)['id']
        else:
            print(res.content)

#### ListUsers

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def listUsers(params=None):
    res = get(path='Users',params=params)
    if res.status_code == 200:
        users = json.loads(res.text)
        return users

#### HasUserByUsername

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def hasUserByUsername(userName=None):
    if userName:
        users = listUsers(f'filter=userName eq "{userName}"')['Resources']
        for u in users:
            if u['userName'] == userName:
                return True
    return False

#### GetUserIDByUsername

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def getUserIDByUsername(userName=None):
    if userName:
        users = listUsers(f'filter=userName eq "{userName}"')['Resources']
        for u in users:
            if u['userName'] == userName:
                return u['id']

#### GetUser

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def getUser(user_id=None):
    if user_id:
        res = get(path=f"Users/{user_id}")
        if res.status_code == 200:
            return json.loads(res.text)

#### ReplaceUser

https://datatracker.ietf.org/doc/html/rfc7644#section-3.5.1

In [ ]:
def replaceUser(user_id=None,userName=None,familyName=None,givenName=None,displayName=None,email=None,
               preferredLanguage="en-US",locale="en-US",timezone="America/Sao_Paulo",active=True):
    if user_id and userName and familyName and givenName and displayName and email:
        data = {
            "id": f"{user_id}",
            "userName": f"{userName}",
            "name": {
                "familyName": f"{familyName}",
                "givenName": f"{givenName}",
            },
            "displayName": f"{displayName}",
            "emails": [
                {
                    "value": f"{email}",
                    "type": "work",
                    "primary": True
                }
            ],
            "preferredLanguage": f"{preferredLanguage}",
            "locale": f"{locale}",
            "timezone": f"{timezone}",
            "active": f"{active}",
        }
        res = put(path=f"Users/{user_id}", data=json.dumps(data))
        if res.status_code == 200:
            return json.loads(res.text)['id']
        else:
            print(res.content)

#### UpdateUser - I need improve this...

https://datatracker.ietf.org/doc/html/rfc7644#section-3.5.2

In [ ]:
def updateUser(user_id=None, data=None):
    return json.loads(patch(path=f"Users/{user_id}", data=data).text)

#### DeleteUser

https://datatracker.ietf.org/doc/html/rfc7644#section-3.6

In [ ]:
def deleteUser(user_id=None):
    res = delete(path=f"Users/{user_id}")
    if res.status_code == 204:
        return True
    return False

### Groups

#### CreateGroup

https://datatracker.ietf.org/doc/html/rfc7644#section-3.3

In [ ]:
def createGroup(groupName=None):
    if groupName:
        data = {"displayName": f"{groupName}"}
        res = post(path=f"Groups", data=json.dumps(data))
        if res.status_code == 201:
            return json.loads(res.text)['id']
        else:
            print(res.content)

#### ListGroups

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def listGroups(params=None):
    res = get(path='Groups',params=params)
    if res.status_code == 200:
        groups = json.loads(res.text)
        return groups

#### HasGroupByName

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def hasGroupByName(groupName=None):
    if groupName:
        groups = listGroups(f'filter=displayName eq "{groupName}"')['Resources']
        for g in groups:
            if g['displayName'] == groupName:
                return True
    return False

#### GetGroupIDByName

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def getGroupIBByName(groupName=None):
    if groupName:
        groups = listGroups(f'filter=displayName eq "{groupName}"')['Resources']
        for g in groups:
            if g['displayName'] == groupName:
                return g['id']

#### GetGroup

https://datatracker.ietf.org/doc/html/rfc7644#section-3.4.1

In [ ]:
def getGroup(group_id=None):
    if group_id:
        res = get(path=f"Groups/{group_id}")
        if res.status_code == 200:
            return json.loads(res.text)

#### UpdateGroup

https://datatracker.ietf.org/doc/html/rfc7644#section-3.5.2

In [ ]:
def updateGroup(group_id=None, operation=None, members=None):
    if group_id and operation and members:
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                    "value":[{"value": f"{member}"} for member in members]
                }
            ]
        }
        res = patch(path=f"Groups/{group_id}", data=json.dumps(data))
        if res.status_code == 204:
            return True
        else:
            print(res.content)
            return False

In [ ]:
def updateGroupFix(group_id=None, operation=None, members=[]):
    if len(members) > 0:
        data_values = [{"value": f"{member}"} for member in members]
    else:
        data_values = [{"value": ""}]
        
    if group_id and operation:
        print(f"Updating {group_id}")
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                    "value": data_values
                }
            ]
        }
        res = patch(path=f"Groups/{group_id}", data=json.dumps(data))
        if res.status_code == 204:
            return True
        else:
            print(res.content)
            return False

#### DeleteGroup

https://datatracker.ietf.org/doc/html/rfc7644#section-3.6

In [ ]:
def deleteGroup(group_id=None):
    res = delete(path=f"Groups/{group_id}")
    if res.status_code == 204:
        return True
    return False

### CreateOrUpdateUser

Method for creating or updating a user by SCIM provisioning

In [ ]:
def CreateOrUpdateUser(member=None):
    print(f"{member['displayName']} => {member['active']}")
    if not hasUserByUsername(member['userName']):
        print(f"--> Creating user {member['userName']} -> {member['displayName']}")
        ID = createUser(userName=member['userName'],
                        familyName=member['familyName'],
                        givenName=member['givenName'],
                        displayName=member['displayName'],
                        email=member['email'],
                        preferredLanguage="en-US",
                        locale="en-US",
                        timezone="America/Sao_Paulo",
                        active=member['active'])
        if ID:
            print(f"----> User created: {ID}")
        else:
            print("----> User create failed")
    else:
        ID = getUserIDByUsername(member['userName'])
        print(f"--> Updating user {member['userName']} -> {member['displayName']} -> {ID}")  
        if replaceUser(user_id=ID,
                       userName=member['userName'],
                       familyName=member['familyName'],
                       givenName=member['givenName'],
                       displayName=member['displayName'],
                       email=member['email'],
                       preferredLanguage="en-US",
                       locale="en-US",
                       timezone="America/Sao_Paulo",
                       active=member['active']):
            print("----> User updated")
        else:
            print("----> User update failed")
    return ID

### listOfUsernamesToIDS

Helper to create a list of IDs from a list of usernames. Need a dictionary to do the black magic to work

In [ ]:
def listOfUsernamesToIDS(usernames=None, usernames_dict=None):
    IDs=[]
    for username in usernames:
        if username in usernames_dict:
            IDs.append(usernames_dict[username])
    return IDs

## Create Users and Groups

In [ ]:
groups_with_members=[]
groups_with_members.append(
    {
        'group_name': 'group1',
        'group_members':[
            {'userName': 'user1', 'familyName': 'user1', 'givenName': 'user1', 'displayName': 'user1', 'email': 'user1@foo.bar', 'preferredLanguage': 'en-US', 'locale': 'en-US', 'timezone': 'America/Sao_Paulo', 'active': True},
            {'userName': 'user2', 'familyName': 'user2', 'givenName': 'user2', 'displayName': 'user2', 'email': 'user2@foo.bar', 'preferredLanguage': 'en-US', 'locale': 'en-US', 'timezone': 'America/Sao_Paulo', 'active': True},
        ]
    }
)
groups_with_members.append(
    {
        'group_name': 'group2',
        'group_members':[
            {'userName': 'user1', 'familyName': 'user1', 'givenName': 'user1', 'displayName': 'user1', 'email': 'user1@foo.bar', 'preferredLanguage': 'en-US', 'locale': 'en-US', 'timezone': 'America/Sao_Paulo', 'active': True},
            {'userName': 'user3', 'familyName': 'user3', 'givenName': 'user3', 'displayName': 'user3', 'email': 'user3@foo.bar', 'preferredLanguage': 'en-US', 'locale': 'en-US', 'timezone': 'America/Sao_Paulo', 'active': True},
        ]
    }
)


### Run and Create

#### members_dict

Dict to store username => id

```{'username1': 'id1', 'username2': 'id2', 'username..n': 'id..n'}```

In [ ]:
members_dict={}

#### members_unique

List of members dict

``` [{member1}, {member2}, {member..n}] ```

In [ ]:
members_unique=[]

#### Populate members_dict and members_unique

In [ ]:
total_processed=0
for group in groups_with_members:
    if group['group_members']:
        for member in group['group_members']:
            total_processed=total_processed+1
            if not member in members_unique:
                members_unique.append(member)
                memberID = getUserIDByUsername(member['userName'])
                members_dict[member['userName']]=memberID
print(f"Groups: {len(groups_with_members)} | Processed members: {total_processed} | Unique members: {len(members_unique)}")



#### Create or Update unique members

In [ ]:
cont=0
for member in members_unique:
    cont = cont +1
    print(f">{cont}/{len(members_unique)}")
    CreateOrUpdateUser(member)

In [ ]:
step=0
for group in groups_with_members:
    step=step+1
    group_name = group['group_name'].upper()
    print(f"({step}/{len(groups_with_members)}) Working in the group: {group_name}")
    members=[]
    if group['group_members']:
        for member in group['group_members']:
            members.append(member['userName'])
    if members:
        if hasGroupByName(group_name):
            print(f"--> Group exists")
            IDs = listOfUsernamesToIDS(members,members_dict)
            GroupID = getGroupIBByName(group_name)
            if updateGroup(GroupID,"replace",IDs):
                print("----> Group members updated")
            else:
                print("----> Group members update failed")
        else:
            print(f"--> Creating group")
            Group_ID = createGroup(group_name)
            print(f"----> Group created: {Group_ID}")
            time.sleep(5)            

            if Group_ID:
                print("----> Group created")
                IDs = listOfUsernamesToIDS(members,members_dict)
                if updateGroup(GroupID,"replace",IDs):
                    print("------> Group members updated")
                else:
                    print("------> Group members update failed")
            else:
                print("----> Group create failed")


# Show me the ~~code~~ problem!

### 1. Correct, but doesn't work

When we need to update a group with an empty list using SCIM, the correct payload is:

```json
{
    "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
    "Operations":[
        {
            "op": "replace",
            "path": "members",
            "value":[]
        }
    ]
}
```

The `value` must be an empty list. When we perform this action on the AWS IAM Identity Center SCIM Endpoint with the correct payload, it returns an HTTP status code of 204 but does not remove users from the group. See below:

In [ ]:
def updateGroup1(group_id=None, operation=None, members=None):
    if group_id and operation:
        print(f"Updating {group_id}")
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                    "value":[{"value": f"{member}"} for member in members]
                }
            ]
        }
        print(json.dumps(data))
        res = patch(path=f"Groups/{group_id}", data=json.dumps(data))
        if res.status_code == 204:
            print(res)
            print(res.content)
            return True
        else:
            print(res)
            print(res.content)
            return False

updateGroup1(getGroupIBByName('GROUP1'),"replace",[])


**See, the group still contains the users.**

![](img/poc01.png)

### 2. Correct, but doesn't work (Other case)

From [AWS IAM Identity Center Documentation](https://docs.aws.amazon.com/singlesignon/latest/developerguide/patchgroup.html):

> In the value field, provide a list of objects containing the value of the user id. Multiple members can be removed at a time. If the value field contains an empty list or is not provided, all of the path’s members will be removed.

Then, let's test a new payload without `value`:

```json
{
    "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
    "Operations":[
        {
            "op": "replace",
            "path": "members",
        }
    ]
}
```

Again, when we execute this action on the AWS IAM Identity Center SCIM Endpoint with another correct payload, it returns an HTTP status code of 204 but does not remove users from the group. See below:

In [ ]:
def updateGroup3(group_id=None, operation=None, members=[]):
    if len(members) > 0:
        data_values = [{"value": f"{member}"} for member in members]
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                    "value": data_values
                }
            ]
        }
    else:
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                }
            ]
        }
        
    if group_id and operation:
        print(f"Updating {group_id}")
        print(json.dumps(data))
        res = patch(path=f"Groups/{group_id}", data=json.dumps(data))
        if res.status_code == 204:
            print(res)
            print(res.content)
            return True
        else:
            print(res)
            print(res.content)
            return False

updateGroup3(getGroupIBByName('GROUP1'),"replace",[])

**See, the same result, group still contains the users.**

![](img/poc01.png)

### 3. Wrong, but work

Now we will proceed with the incorrect payload:

```json
{
    "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
    "Operations":[
        {
            "op": "replace",
            "path": "members",
            "value":[{"value": ""}]
        }
    ]
}
```

Now, the AWS IAM Identity Center SCIM Endpoint will return an HTTP status code of 400 along with an error message, but it removes users from the group. See below:

In [ ]:
def updateGroup2(group_id=None, operation=None, members=[]):
    if len(members) > 0:
        data_values = [{"value": f"{member}"} for member in members]
    else:
        data_values = [{"value": ""}]
        
    if group_id and operation:
        print(f"Updating {group_id}")
        data = {
            "schemas":["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
            "Operations":[
                {
                    "op": f"{operation}",
                    "path": "members",
                    "value": data_values
                }
            ]
        }
        print(json.dumps(data))
        res = patch(path=f"Groups/{group_id}", data=json.dumps(data))
        if res.status_code == 204:
            print(res)
            print(res.content)
            return True
        else:
            print(res)
            print(res.content)
            return False

updateGroup2(getGroupIBByName('GROUP1'),"replace",[])

**See now, although an error was returned, the group is empty.**

![](img/poc02.png)

### Just one more complaint, if I may


The `GET /Groups/<ID>` method returns the `members` attribute as empty. According to [RFC7643](https://datatracker.ietf.org/doc/html/rfc7643#section-8.4), it should return this JSON:

```json
 {
     "schemas": ["urn:ietf:params:scim:schemas:core:2.0:Group"],
     "id": "e9e30dba-f08f-4109-8486-d5c6a331660a",
     "displayName": "Tour Guides",
     "members": [
       {
         "value": "2819c223-7f76-453a-919d-413861904646",
         "$ref":
   "https://example.com/v2/Users/2819c223-7f76-453a-919d-413861904646",
         "display": "Babs Jensen"
       },
       {
         "value": "902c246b-6245-4190-8e05-00816be7344a",
         "$ref":
   "https://example.com/v2/Users/902c246b-6245-4190-8e05-00816be7344a",
         "display": "Mandy Pepperidge"
       }
     ],
     "meta": {
       "resourceType": "Group",
       "created": "2010-01-23T04:56:22Z",
       "lastModified": "2011-05-13T04:42:34Z",
       "version": "W\/\"3694e05e9dff592\"",
       "location":
   "https://example.com/v2/Groups/e9e30dba-f08f-4109-8486-d5c6a331660a"
     }
   }
```